In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import os
import re
from collections import defaultdict
import pprint as pp

from tqdm import tqdm
RE_FNAME = re.compile(r'LP\-(?P<idx>[0-9])\-STRM\-[0-9]\-(?P<pair>[A-Z]+).*\.csv')

from utils import GCStorage
from constants import CREDENTIAL_PATH, ALL_DAYS

import pandas as pd

import shutil
import csv
import numpy as np
from subprocess import call
import pickle

import datetime
from dateutil.parser import parse
from pathlib import Path

from utils import *

In [9]:
time_interval = 10

In [10]:
pickle_path = Path('/sailhome/jingbo/CXR_RELATED/temp_store/pickled_data')
processed_30s_path = Path(f'/sailhome/jingbo/CXR_RELATED/temp_store/processed_{time_interval}s_data')

processed_30s_path.mkdir(parents=True, exist_ok=True)

In [11]:
storage = GCStorage('FX_Trading', 'integral_data', CREDENTIAL_PATH)

In [12]:
candle_interval = datetime.timedelta(seconds=time_interval)

In [13]:
for day in tqdm(ALL_DAYS):
    # print(day)
    # for day in ['2019-02-05']:
    for pair in ['AUDUSD', 'EURUSD', 'GBPUSD', 'USDCAD', 'USDCHF']:
        # print(pair)
        for lp in range(1, 6):
            # print(lp)
            
            try:
                start_time = parse(day)
                end_time = start_time + datetime.timedelta(days=1)
                whole_day = start_time.replace(minute=0, hour=0,
                                               second=0, microsecond=0)
                nice_date = whole_day.strftime('%Y%m%d')
                fname = f'{nice_date}-{pair}-{lp}.pickle'
                pickled_file = pickle_path / pair / nice_date / fname

                day_df = pickle.load(open(pickled_file, 'rb'))

                candles = generate_candles(day_df, f'{time_interval}s', 'mid price', get_df=False)

                candle_df = pad_candle(candles, candle_interval,
                                       start_time, end_time)

                processed_file = processed_30s_path / pair / nice_date / fname
                os.makedirs(processed_30s_path / pair / nice_date, exist_ok=True)
                pickle.dump(candle_df, open(processed_file, 'wb'))

                cloud_path = Path(f'processed_{time_interval}s_data') / pair / nice_date / fname
                storage.upload(processed_file, cloud_path)
            except:
                print(f'Failed at {day} {pair} {lp}')


 44%|████▍     | 11/25 [05:59<07:56, 34.03s/it]

Failed at 2019-02-14 AUDUSD 2


100%|██████████| 25/25 [14:40<00:00, 40.50s/it]


In [ ]:
candle_df